1.   Davi Alves
2.   Kawê Gomes
3. Thiago Machado




In [ ]:
from functools import partial
from typing import List, Tuple, Union

import numpy as np
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display

In [ ]:
X = +1
O = -1
EMPTY_SPACE = 0

INTERFACE_MAPPING = {
    X: 'X',
    O: 'O'
}

In [ ]:
class Interface:
    def __init__(self):
        self.buttons = [
            [
                widgets.Button(
                    description='',
                    layout=Layout(width='100px', height='100px')
                )
                for _ in range(3)
            ]
            for _ in range(3)
        ]

        self.board_widget = VBox([HBox(row) for row in self.buttons])

        self.__player = X
        self.link_positions()

    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the board

        :param pos: row, column of button on grid
        :param button: clicked button
        """
        if not button.description:
            button.description = INTERFACE_MAPPING.get(self.__player, '')
            self.__player *= -1  # invert player

    def link_positions(self):
        """Link clicks on buttons"""
        for i, row in enumerate(self.buttons):
            for j, button in enumerate(row):
                button.on_click(partial(self.on_select_position, (i, j)))

    def disable_buttons(self):
        """Disable buttons to avoid clicks after end game"""
        for row in self.buttons:
            for button in row:
                button.disabled = True


    def start(self):
        """Display board interface"""
        display(self.board_widget)

In [ ]:
interface = Interface()
interface.start()

In [ ]:
board = np.zeros(shape=(3, 3), dtype=int)
board[1, 0] = 1
board

array([[0, 0, 0],
       [1, 0, 0],
       [0, 0, 0]])

In [ ]:
np.diag(board)

array([0, 1, 0])

In [ ]:
if sum(np.diag(board)) == 3:
    print(True)
else:
    print(False)

False


In [ ]:
def get_game_status(board: np.ndarray) -> Tuple[bool, Union[int, None]]:
    """Check if game ended and who is the winner

    :param board:
    :return: if its a game over, who is the winner
    """
    # verifica se alguma linha soma 3 ou -3
    for row in board:
        if sum(row) in {3, -3}:
            return True, row[0]

    # verifica se alguma coluna soma 3 ou -3
    for col in board.T:
        if sum(col) in {3, -3}:
            return True, col[0]

    # verifica se diagonal principal soma 3 ou -3
    if sum(np.diag(board)) in {3, -3}:
        return True, board[0, 0]

    # verifica se diagonal secundária soma 3 ou -3
    if sum(np.diag(np.fliplr(board))) in {3, -3}:
        return True, board[0, 2]

    # verifica se deu velha.
    if (board != 0).all():
        return True, None

    return False, None


board = np.zeros(shape=(3, 3), dtype=int)
board[1] = 1
print(board)
get_game_status(board)

[[0 0 0]
 [1 1 1]
 [0 0 0]]


(True, 1)

In [ ]:
board.shape

(3, 3)

In [ ]:
def get_possible_moves(board: np.ndarray, player: int = X) -> List[np.ndarray]:
    """Get next possible moves by some player
    """
    moves = []
    for row in range(board.shape[0]):
        for col in range(board.shape[1]):
            if board[row, col] == EMPTY_SPACE:
                new_board = board.copy()
                new_board[row, col] = player
                moves.append(new_board)

    return moves
get_possible_moves(board)



[array([[1, 0, 0],
        [1, 0, 0],
        [0, 0, 0]]),
 array([[0, 1, 0],
        [1, 0, 0],
        [0, 0, 0]]),
 array([[0, 0, 1],
        [1, 0, 0],
        [0, 0, 0]]),
 array([[0, 0, 0],
        [1, 1, 0],
        [0, 0, 0]]),
 array([[0, 0, 0],
        [1, 0, 1],
        [0, 0, 0]]),
 array([[0, 0, 0],
        [1, 0, 0],
        [1, 0, 0]]),
 array([[0, 0, 0],
        [1, 0, 0],
        [0, 1, 0]]),
 array([[0, 0, 0],
        [1, 0, 0],
        [0, 0, 1]])]

In [ ]:
AI_PLAYER = X
HUMAN_PLAYER = O
MAX_N_MOVES = 9


def get_score(winner: int, n_moves: int) -> int:
    """Get how well was the game for the AI:
        - win faster is better than win slower
        - lose slower is better than win faster
        - draw is a intermediary result
    """
    if winner == AI_PLAYER:
        return MAX_N_MOVES + 1 - n_moves
    elif winner == HUMAN_PLAYER:
        return n_moves - (MAX_N_MOVES + 1)
    else:
        return 0


In [ ]:
def mini_max(board, player=AI_PLAYER, n_moves=0):
    is_over, winner = get_game_status(board)
    if is_over:
        return get_score(winner, n_moves), board

    possibilities = []
    for move in get_possible_moves(board, player):
        score, _ = mini_max(move, -player, n_moves + 1)
        possibilities.append((score, move))

    if player == AI_PLAYER:
        return max(possibilities, key=lambda x: x[0])
    else:
        return min(possibilities, key=lambda x: x[0])


In [ ]:
board = np.zeros(shape=(3, 3), dtype=np.int8)
_, new_board = mini_max(board, AI_PLAYER)
new_board


array([[1, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int8)

In [ ]:
new_board = new_board.copy()
new_board[1, 0] = -1
_, new_board = mini_max(new_board, AI_PLAYER)
new_board

array([[ 1,  1,  0],
       [-1,  0,  0],
       [ 0,  0,  0]], dtype=int8)

In [ ]:
END_GAME_MESSAGES = {
    AI_PLAYER: 'AI won!',
    HUMAN_PLAYER: 'You won!',
    EMPTY_SPACE: "It is a draw"
}


class TicTacToeAI(Interface):
    def __init__(self, ai_starts=False):
        super().__init__()
        self.board = np.zeros(shape=(3, 3), dtype=np.int8)
        if ai_starts:
            self.board = min_max(self.board, AI_PLAYER)[1].copy()
            self.update()


    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the board
        It calls minmax algorithm after each user move

        :param pos: row, column of button on grid
        :param button: clicked button"""
        if not button.description:
            self.board[pos] = HUMAN_PLAYER
            self.board = mini_max(self.board, AI_PLAYER)[1].copy()

            is_over, winner = get_game_status(self.board)
            if is_over:
                self.disable_buttons()
                print(END_GAME_MESSAGES.get(winner))

            self.update()

    def update(self):
        """Update interface from virtual board"""
        for i, row in enumerate(self.board):
            for j, item in enumerate(row):
                self.buttons[i][j].description = INTERFACE_MAPPING.get(item, '')


In [ ]:
game = TicTacToeAI(ai_starts=False)
game.start()

AI won!
